In [1]:
import spacy
import requests
import pandas as pd
from spacy.lang.en.examples import sentences
from spacy import displacy
import matplotlib.pyplot as plt

In [2]:
# Load the spacy model
nlp = spacy.load('en_core_web_md')

In [3]:
text = "I work as a software engineer for Goldman Sachs in New York City."
displacy.render(nlp(text), style='ent', jupyter=True)

In [6]:
# read xlsx file as a df
df = pd.read_excel('../data/BookingDotCom_HotelReviews.xlsx')
df.head()

,Hotel_Name,Hotel_Address,Review_Count,Non_Review_Scoring_Count,Average_Hotel_Score,Review_Date,Reviewer_Nationality,Positive_Comments,Negative_Comments,Total_Reviewer_Reviews,Reviewer_Score,Tags
0,Hotel Arena,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,1403,194,7.7,2017-08-03,Russia,Only the park outside of the hotel was beauti...,I am so angry that i made this post available...,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double..."
1,Hotel Arena,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,1403,194,7.7,2017-08-03,Ireland,No real complaints the hotel was great great ...,No Negative,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double..."
2,Hotel Arena,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,1403,194,7.7,2017-07-31,Australia,Location was good and staff were ok It is cut...,Rooms are nice but for elderly a bit difficul...,9,7.1,"[' Leisure trip ', ' Family with young childre..."
3,Hotel Arena,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,1403,194,7.7,2017-07-31,United Kingdom,Great location in nice surroundings the bar a...,My room was dirty and I was afraid to walk ba...,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex..."
4,Hotel Arena,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,1403,194,7.7,2017-07-24,New Zealand,Amazing location and building Romantic setting,You When I booked with your company on line y...,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St..."


In [9]:
# normalise the columns, make everything lowercase
df.columns = df.columns.str.lower()
df.columns

Index(['hotel_name', 'hotel_address', 'review_count',
       'non_review_scoring_count', 'average_hotel_score', 'review_date',
       'reviewer_nationality', 'positive_comments', 'negative_comments',
       'total_reviewer_reviews', 'reviewer_score', 'tags'],
      dtype='object')

In [10]:
# make 2 new df's, one for positive reviews and one for negative reviews
df_pos = df[df['reviewer_score'] > 7]
df_neg = df[df['reviewer_score'] < 4]
df_pos.head()

,hotel_name,hotel_address,review_count,non_review_scoring_count,average_hotel_score,review_date,reviewer_nationality,positive_comments,negative_comments,total_reviewer_reviews,reviewer_score,tags
1,Hotel Arena,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,1403,194,7.7,2017-08-03,Ireland,No real complaints the hotel was great great ...,No Negative,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double..."
2,Hotel Arena,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,1403,194,7.7,2017-07-31,Australia,Location was good and staff were ok It is cut...,Rooms are nice but for elderly a bit difficul...,9,7.1,"[' Leisure trip ', ' Family with young childre..."
7,Hotel Arena,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,1403,194,7.7,2017-07-17,United Kingdom,Good location Set in a lovely park friendly s...,Apart from the price for the brekfast Everyth...,1,10.0,"[' Leisure trip ', ' Couple ', ' Duplex Double..."
9,Hotel Arena,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,1403,194,7.7,2017-07-08,Norway,The room was big enough and the bed is good T...,The aircondition makes so much noise and its ...,1,7.9,"[' Leisure trip ', ' Couple ', ' Large King Ro..."
10,Hotel Arena,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,1403,194,7.7,2017-07-07,United Kingdom,Rooms were stunningly decorated and really sp...,Nothing all great,2,10.0,"[' Leisure trip ', ' Group ', ' Duplex Double ..."


In [11]:
positive_features = dict()
# iterate through the positive reviews and extract the features from column positive_comments
for index, row in df_pos.iterrows():
    positive = row['positive_comments']
    doc = nlp(positive)
    # if pos is noun, or adjective add it to the dictionary
    for token in doc:
        if token.pos_ == 'NOUN' or token.pos_ == 'ADJ':
            if token.text in positive_features:
                positive_features[token.text] += 1
            else:
                positive_features[token.text] = 1

# sort the dictionary by the values
positive_sorted = sorted(positive_features.items(), key=lambda x: x[1], reverse=True)
positive_sorted

KeyboardInterrupt: 